In [1]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

pygame 2.6.0 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from custom_env import CustomEnv
from wrappers import FullyObsSB3MLPWrapper
from callbacks import EvalSaveCallback

C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\13769\anaconda3\envs\mini-grid-generation\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
# Define the session directory
session_dir = r"./experiments/door_key"
os.makedirs(session_dir, exist_ok=True)

# Set the log directory within the session directory
log_dir = os.path.join(session_dir, "logs")
os.makedirs(log_dir, exist_ok=True)

# Set training parameters
max_train_episode_steps = 1000
max_eval_episode_steps = 500
total_timesteps = int(2e5)
eval_deterministic = False
eval_freq = int(5e3)
n_eval_episodes = 10
model_save_path = os.path.join(session_dir, "latest_model")

In [4]:
# Initialize the environment and wrapper
train_env = CustomEnv(
    txt_file_path=r'./maps/simple_test_door_key.txt',
    display_size=6,
    display_mode="random",
    random_rotate=True,
    random_flip=True,
    custom_mission="Find the key and open the door.",
    max_steps=max_train_episode_steps
)

# Wrap the environment with FullyObsSB3MLPWrapper
train_env = FullyObsSB3MLPWrapper(train_env)

# Use DummyVecEnv and VecMonitor to create and wrap the environment
train_env = DummyVecEnv([lambda: train_env])  # Vectorize the environment
train_env = VecMonitor(train_env)  # Track episode statistics such as rewards

In [5]:
# Initialize the environment and wrapper
eval_env = CustomEnv(
    txt_file_path=r'./maps/simple_test_door_key.txt',
    display_size=6,
    display_mode="middle",
    random_rotate=False,
    random_flip=False,
    custom_mission="Find the key and open the door.",
    max_steps=max_eval_episode_steps
)

# Wrap the environment with FullyObsSB3MLPWrapper
eval_env = FullyObsSB3MLPWrapper(eval_env)

# Use DummyVecEnv and VecMonitor to create and wrap the environment
eval_env = DummyVecEnv([lambda: eval_env])  # Vectorize the environment
eval_env = VecMonitor(eval_env)  # Track episode statistics such as rewards

In [6]:
# Load or create a new model
if os.path.exists(f"{model_save_path}.zip"):
    model = PPO.load(model_save_path, env=train_env)
    print("Loaded model from saved path.")
else:
    model = PPO("MlpPolicy", train_env, verbose=1)
    print("Initialized new model.")

Using cuda device
Initialized new model.


In [7]:
# Create EvalSaveCallback
eval_save_callback = EvalSaveCallback(
    eval_env=eval_env,
    log_dir=log_dir,
    eval_freq=eval_freq,
    n_eval_episodes=n_eval_episodes,
    deterministic=eval_deterministic,
    verbose=1,
)

In [ ]:
# Train the model and log performance with the custom callback
model.learn(total_timesteps=total_timesteps, callback=eval_save_callback, progress_bar=True)

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 472      |
|    ep_rew_mean     | 0.55     |
| time/              |          |
|    fps             | 396      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 401         |
|    ep_rew_mean          | 0.629       |
| time/                   |             |
|    fps                  | 355         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013241606 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.94       |
|    explained_variance   | 0.0813      |
|    learning_rate        | 0.

Evaluating model...

Evaluation: Mean reward: 0.46 +/- 0.31

Saved latest model to ./experiments/door_key\logs\latest_model.zip

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.47        |
|    ep_rew_mean          | 0.994       |
| time/                   |             |
|    fps                  | 293         |
|    iterations           | 76          |
|    time_elapsed         | 530         |
|    total_timesteps      | 155648      |
| train/                  |             |
|    approx_kl            | 0.020214964 |
|    clip_fraction        | 0.193       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.787      |
|    explained_variance   | 0.84        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.018       |
|    n_updates            | 750         |
|    policy_gradient_loss | -9.79e-05   |
|    value_loss           | 0.00139     |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 14.5    

Evaluating model...

Evaluation: Mean reward: 0.99 +/- 0.00

Saved latest model to ./experiments/door_key\logs\latest_model.zip

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 15.4        |
|    ep_rew_mean          | 0.986       |
| time/                   |             |
|    fps                  | 292         |
|    iterations           | 79          |
|    time_elapsed         | 553         |
|    total_timesteps      | 161792      |
| train/                  |             |
|    approx_kl            | 0.074879035 |
|    clip_fraction        | 0.274       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.653      |
|    explained_variance   | 0.823       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0126      |
|    n_updates            | 780         |
|    policy_gradient_loss | -0.00284    |
|    value_loss           | 0.000992    |
-----------------------------------------
